<a href="https://colab.research.google.com/github/Strincience/Audio-Analysis-Project/blob/main/Audio_Synthesis_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An Audio-to-text transcriber. A model that transcribes audio to text. (Obviously 🙂)

# Importing and installing the Neccessary Libraries

In [ ]:
!pip install torchaudio transformers librosa -q
!pip install rake-nltk -q
!pip install nltk -q

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize


# !pip install pip-progress-bar -q
# !PIP_PROGRESS_BAR=on pip install torchaudio transformers librosa -q
# !PIP_PROGRESS_BAR=on pip install rake-nltk -q
# !PIP_PROGRESS_BAR=on pip install nltk -q



# from tqdm.auto import tqdm; import subprocess; p = subprocess.Popen("pip install torchaudio transformers librosa -q", shell=True); [tqdm(total=1, desc="Installing") for _ in [p.wait()]]
# from tqdm.auto import tqdm; import subprocess; p = subprocess.Popen("pip install rake-nltk -q", shell=True); [tqdm(total=1, desc="Installing") for _ in [p.wait()]]
# from tqdm.auto import tqdm; import subprocess; p = subprocess.Popen("pip install nltk -q", shell=True); [tqdm(total=1, desc="Installing") for _ in [p.wait()]]


In [ ]:
!pip install keybert -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.5 MB/s eta 0:00:00


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import os

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Loading the processor and the model

In [ ]:
# # Loading the processor and the model
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# # Sending the model to te device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)


# Initialize processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [ ]:
# ## Defining the folder path
# # import os

# folder_path = r"/content/drive/MyDrive/Colab Notebooks/Audio Recordings test"

# # for file in os.listdir(folder_path):
# #     if file.endswith(".flac") or file.endswith(".wav") or file.endswith(".m4a"):
# #         full_path = os.path.join(folder_path, file)
# #         print(f"🔊 Transcribing {file}...")
# #         print("📜", transcribe_audio(full_path))
# #         print("-" * 40)



In [ ]:
def load_audio(path):

  # Loading the audio file
  waveform, sample_rate = torchaudio.load(path)
  waveform = waveform.to(device)

  # Resampling to 16000 kHz if needed
  if sample_rate != 16000:
    print("⚠️ The sample Rate {sample_rate} is not 16000, resampling...")
    resampler = torchaudio.transforms.Resample(sample_rate, 16000).to(device)
    waveform = resampler(waveform)
  return waveform.squeeze()
  # input_values = processor(waveform,
  #                          sampling_rate=sample_rate,
  #                          return_tensors="pt",
  #                          padding="longest").input_values.to(device)



In [ ]:
# import sounddevice as sd
# import soundfile as sf
# import numpy as np

# ## Function to record

# def record(duration = 10, sample_rate = 16000, filename = "recording.wav"):
#   print(f"Recording for {duration} seconds...")
#   recording = sd.rec(int(duration * sample_rate), samplerate = sample_rate, channels = 1)
#   sd.wait()
#   sf.write(filename, recording, sample_rate)
#   print(f"Recording saved as {filename}")
#   return filename


# Loading the audio and transcribing it

#Generating audio automatically with Eleven labs API

In [ ]:
pip install requests -q


## Getting the Voices and voice ID from ELvenlabs

In [ ]:
import requests

API_KEY = "sk_01174f957269d31618af2acddb60f6b193365dfb8551591d"

response = requests.get(
    "https://api.elevenlabs.io/v1/voices",
    headers={"xi-api-key": API_KEY}
)

voices = response.json()["voices"]

for voice in voices:
    print(f"Name: {voice['name']}, ID: {voice['voice_id']}")


Name: Rachel, ID: 21m00Tcm4TlvDq8ikWAM
Name: Clyde, ID: 2EiwWnXFnvU5JabPnv8n
Name: Roger, ID: CwhRBWXzGAHq8TQ4Fs17
Name: Sarah, ID: EXAVITQu4vr4xnSDxMaL
Name: Laura, ID: FGY2WhTYpPnrIDTdsKH5
Name: Thomas, ID: GBv7mTt0atIp3Br8iCZE
Name: Charlie, ID: IKne3meq5aSn9XLyUdCD
Name: George, ID: JBFqnCBsd6RMkjVDRZzb
Name: Callum, ID: N2lVS1w4EtoT3dr4eOWO
Name: River, ID: SAz9YHcvj6GT2YYXdXww
Name: Harry, ID: SOYHLrjzK2X1ezoPC6cr
Name: Liam, ID: TX3LPaxmHKxFdv7VOQHJ
Name: Alice, ID: Xb7hH8MSUJpSbSDYk0k2
Name: Matilda, ID: XrExE9yKIg1WjnnlVkGX
Name: Will, ID: bIHbv24MWmeRgasZH58o
Name: Jessica, ID: cgSgspJ2msm6clMCkdW9
Name: Eric, ID: cjVigY5qzO86Huf0OWal
Name: Chris, ID: iP95p4xoKVk53GoZ742B
Name: Brian, ID: nPczCjzI2devNBz1zQrb
Name: Daniel, ID: onwK4e9ZLuTAKqWW03F9
Name: Lily, ID: pFZP5JQG7iQjIQuC4Bku
Name: Bill, ID: pqHfZKP75CvOlQylNhV4
Name: Olufunmilola - African Female with Nigerian Accent - Yoruba, ID: 9Dbo4hEvXQ5l7MXGZFQA
Name: Yomi - Young African Nigerian Youruba Accent, ID: x86DtpnPPu

In [ ]:
import requests
import time
import random

API_KEY = "sk_01174f957269d31618af2acddb60f6b193365dfb8551591d"

# To randomly pick from the available list of voices.
# Assuming 'voices' list is available from the previous cell's execution
VOICE_ID = random.choice(voices)['voice_id']

text_list = [""" The interface looks clean, but it took me a while to figure out how to navigate the settings """,

             """I’ve recommended this to three friends already—it’s that good""",
             """Delivery was delayed by two days, which was frustrating, but the product itself is solid""",
             """Honestly, I expected more based on the price tag."""
             """Oh great, another update that fixes everything except the actual problem""",
             """It’s amazing how something can be both overpriced and underwhelming at the same time."""
             """Customer support was super responsive—if you count three days of silence as a response."""

             ]
for i, text in enumerate(text_list):
  timestamp = time.strftime("%Y%m%d%H%M%S%m")
  response = requests.post(
  f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}",
  headers={
      "xi-api-key": API_KEY,
      "Content-Type": "application/json"
  },
  json={
      "text": text,
      "model_id": "eleven_monolingual_v1",
      "voice_settings": {
          "stability": 0.75,
          "similarity_boost": 0.75
      }
  }
)
  with open(f"/content/drive/MyDrive/LibriSpeech/Audio Recordings test/conversation_{timestamp}_{i}.wav", "wb") as f:
    f.write(response.content)

  print(f"✅ Audio saved as conversation_{timestamp}_{i}.wav")
  time.sleep(1)




# response = requests.post(
#     f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}",
#     headers={
#         "xi-api-key": API_KEY,
#         "Content-Type": "application/json"
#     },
#     json={
#         "text": text,
#         "model_id": "eleven_monolingual_v1",
#         "voice_settings": {
#             "stability": 0.75,
#             "similarity_boost": 0.75
#         }
#     }
# )

✅ Audio saved as conversation_2025090216361209_0.wav
✅ Audio saved as conversation_2025090216361409_1.wav
✅ Audio saved as conversation_2025090216361609_2.wav
✅ Audio saved as conversation_2025090216361809_3.wav
✅ Audio saved as conversation_2025090216362109_4.wav


## Let's Hold on with extracting keywords for now

In [ ]:
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from rake_nltk import Rake

# def extract_keywords(transcription):
#   # Initialize Rake
#   rake_nltk_var = Rake()

#   words = word_tokenize(transcription)
#   keywords = rake_nltk_var.run(transcription) # Pass transcription to Rake
#   return f"The Keywords from the above text are {keywords}"

#   # stop_words = set(stopwords.words("english"))
#   # keywords = [w for w in words if w.lower() not in stop_words and w.isalnum()]
#   # return keywords

# extract_keywords(transcription)

NameError: name 'transcription' is not defined

In [ ]:
# import nltk
# try:
#     nltk.data.find('tokenizers/punkt/PY3/english.pickle')
# except nltk.downloader.DownloadError:
#     nltk.download('punkt')
# except LookupError:
#     nltk.download('punkt')

# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from rake_nltk import Rake
from keybert import KeyBERT

def load_audio(path):

  # Loading the audio file
  waveform, sample_rate = torchaudio.load(path)
  waveform = waveform.to(device)

  # Resampling to 16000 kHz if needed
  if sample_rate != 16000:
    print("⚠️ The sample Rate {sample_rate} is not 16000, resampling...")
    resampler = torchaudio.transforms.Resample(sample_rate, 16000).to(device)
    waveform = resampler(waveform)
  return waveform.squeeze()
  # input_values = processor(waveform,
  #                          sampling_rate=sample_rate,
  #                          return_tensors="pt",
  #                          padding="longest").input_values.to(device)


def transcribe(audio_path, processor, model):
  try:
    # load audio
    waveform = load_audio(audio_path)
    # Transcribing the audio
    input_values = processor(waveform,
                             sampling_rate=16000,
                             return_tensors="pt",
                             padding="longest").input_values.to(device)
    # Get Transcription
    with torch.no_grad():
      logits = model(input_values).logits
      # Giving each audio an ID.
      predicted_ids  = torch.argmax(logits, dim = -1)
      # Transcribing the audio into human-readable text.
      transcription = processor.decode(predicted_ids[0])

    return transcription

  except Exception as e:
    print(f"Error Processing {audio_path} : {str(e)}")
    return None

def extract_keywords(transcription):
  # Initialize KeyBERT
  kw_model = KeyBERT()
  keywords = kw_model.extract_keywords(transcription) # Pass transcription to kw_model
  return keywords


if __name__ == "__main__":
  ## The folder_path is the path of the folder that contains the audio, then I will loop through the folder
  folder_path = r"/content/drive/MyDrive/Colab Notebooks/Audio Recordings test"
  # folder_path = r""

  ahe anek
  # folder_path = r""
  if not os.path.exists(folder_path):
    print(f"❌ Folder not found: {folder_path}")
  else:
    print(f"processing files in: {folder_path}")


  for file_name in os.listdir(folder_path):
    if file_name.endswith((".flac", ".wav", "m4a")):
      audio_files = os.path.join(folder_path, file_name)
      print(f"\n🔊 Processing: {folder_path}")

      transcription = transcribe(audio_files, processor, model)
      print("📜", transcription)
      print("-" * 40)

      if transcription: # Only extract keywords if transcription is successful
          keywords = extract_keywords(transcription)
          print(f"The Keywords from the above text are {keywords}")
          print("-" * 40)

processing files in: /content/drive/MyDrive/Colab Notebooks/Audio Recordings test

🔊 Processing: /content/drive/MyDrive/Colab Notebooks/Audio Recordings test
⚠️ The sample Rate {sample_rate} is not 16000, resampling...
📜 THIS IS THE FIRST RECORD DEN HOW RU BEDUIN TO TEST MY TITITHI OR DU TRANSCRIBE OR PROJECT
----------------------------------------


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The Keywords from the above text are [('transcribe', 0.5203), ('record', 0.4565), ('ru', 0.3654), ('titithi', 0.3618), ('test', 0.331)]
----------------------------------------

🔊 Processing: /content/drive/MyDrive/Colab Notebooks/Audio Recordings test
⚠️ The sample Rate {sample_rate} is not 16000, resampling...
📜 THES IS THE SICLANDRY CURDINAL BEDOIN TO TEST MY CITICY PROJECTS
----------------------------------------
The Keywords from the above text are [('projects', 0.5626), ('siclandry', 0.4315), ('curdinal', 0.3542), ('citicy', 0.3536), ('test', 0.2875)]
----------------------------------------

🔊 Processing: /content/drive/MyDrive/Colab Notebooks/Audio Recordings test
⚠️ The sample Rate {sample_rate} is not 16000, resampling...
📜 OUT SO FIND SIN RASHAN LANDING WOD ELSE
----------------------------------------
The Keywords from the above text are [('rashan', 0.5888), ('landing', 0.4168), ('sin', 0.4018), ('wod', 0.3218)]
----------------------------------------

🔊 Processing: /cont

# To confirm that there are flac files in the folder

In [ ]:
# import os

# audio_folder = "/content/drive/MyDrive/Colab Notebooks/Audio Recordings test"
# audio_files = [f for f in os.listdir(audio_folder) if f.endswith((".wav", ".flac", ".m4a"))]

# print(f"Found {len(audio_files)} audio files:")
# print(audio_files[:5])  # Show a few

#Training the model to classify the sentiments using logisitic regression.


#📝 Frozen BERT + Logistic Regression: Sentiment Classifier

In [ ]:
## Installing dependencies

!pip install transformers scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from sklearn.metrics import classification_report

In [ ]:
##Loading the Bert tokeniser and the model

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval() ## Freezing the model

# Movign Project to streamlit

In [ ]:
# !pip install streamlit

# import streamlit as st




In [ ]:
!pip install python-rake

In [ ]:
from nltk.corpus import stopwords
from rake_nltk import Rake